In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import plotly

In [3]:
plotly.offline.init_notebook_mode(connected=True)

In [4]:
import pandas as pd

In [5]:
dataframe = pd.read_csv("titanic.tsv", sep="\t", index_col=0)

dataframe = dataframe[["sex", "age", "fare", "survived"]].dropna()

In [6]:
import numpy as np

In [7]:
import kraft

In [8]:
g = (dataframe["sex"] == "male").astype(int)

g.name = "Gender"

a = dataframe["age"]

a.name = "Age"

f = pd.Series(
    kraft.log_nd_array(
        dataframe["fare"].values, shift_as_necessary_to_achieve_min_before_logging="0<"
    ),
    index=dataframe.index,
)

f.name = "Fare"

s = dataframe["survived"]

s.name = "Survival"

kraft.plot_histogram((g, a, f, s), title="Variables")

In [9]:
p_s1 = (s == 1).sum() / s.size

p_s1

0.40861244019138754

In [10]:
target_dimension_value = 1

fraction_grid_extension = 1/10

n_grid = 64

In [11]:
observation_x_dimension = np.asarray((g, s)).T

n_dimension = observation_x_dimension.shape[1]

p_s1__g = kraft.infer(
    observation_x_dimension,
    target_dimension_value=target_dimension_value,
    dimension_fraction_grid_extensions=(fraction_grid_extension,)*n_dimension,
    dimension_n_grids=(n_grid,)*n_dimension,
    dimension_names=(g.name, s.name),
)

In [12]:
observation_x_dimension = np.asarray((a, s)).T

n_dimension = observation_x_dimension.shape[1]

p_s1__a = kraft.infer(
    observation_x_dimension,
    target_dimension_value=target_dimension_value,
    dimension_fraction_grid_extensions=(fraction_grid_extension,)*n_dimension,
    dimension_n_grids=(n_grid,)*n_dimension,
    dimension_names=(a.name, s.name),
)

In [13]:
observation_x_dimension = np.asarray((f, s)).T

n_dimension = observation_x_dimension.shape[1]

p_s1__f = kraft.infer(
    observation_x_dimension,
    target_dimension_value=target_dimension_value,
    dimension_fraction_grid_extensions=(fraction_grid_extension,)*n_dimension,
    dimension_n_grids=(n_grid,)*n_dimension,
    dimension_names=(f.name, s.name),
)

In [14]:
observation_x_dimension = np.asarray((a, f, s)).T

n_dimension = observation_x_dimension.shape[1]

p_s1__a_f = kraft.infer(
    observation_x_dimension,
    target_dimension_value=target_dimension_value,
    dimension_fraction_grid_extensions=(fraction_grid_extension,)*n_dimension,
    dimension_n_grids=(n_grid,)*n_dimension,
    dimension_names=(a.name, f.name, s.name),
)

3 Dimension Grids:
[[-7.81662999e+00 -6.29599571e+00 -4.77536142e+00 -3.25472714e+00
  -1.73409285e+00 -2.13458565e-01  1.30717572e+00  2.82781001e+00
   4.34844429e+00  5.86907858e+00  7.38971286e+00  8.91034715e+00
   1.04309814e+01  1.19516157e+01  1.34722500e+01  1.49928843e+01
   1.65135186e+01  1.80341529e+01  1.95547871e+01  2.10754214e+01
   2.25960557e+01  2.41166900e+01  2.56373243e+01  2.71579586e+01
   2.86785929e+01  3.01992271e+01  3.17198614e+01  3.32404957e+01
   3.47611300e+01  3.62817643e+01  3.78023986e+01  3.93230329e+01
   4.08436671e+01  4.23643014e+01  4.38849357e+01  4.54055700e+01
   4.69262043e+01  4.84468386e+01  4.99674729e+01  5.14881071e+01
   5.30087414e+01  5.45293757e+01  5.60500100e+01  5.75706443e+01
   5.90912786e+01  6.06119129e+01  6.21325471e+01  6.36531814e+01
   6.51738157e+01  6.66944500e+01  6.82150843e+01  6.97357186e+01
   7.12563529e+01  7.27769871e+01  7.42976214e+01  7.58182557e+01
   7.73388900e+01  7.88595243e+01  8.03801586e+01  8.1900

In [16]:
observation_x_dimension = np.asarray((a, f, s)).T

n_dimension = observation_x_dimension.shape[1]

p_s1__a_f_naive = kraft.infer_assuming_independence(
    observation_x_dimension,
    target_dimension_value=target_dimension_value,
    dimension_fraction_grid_extensions=(fraction_grid_extension,)*n_dimension,
    dimension_n_grids=(n_grid,)*n_dimension,
    dimension_names=(a.name, f.name, s.name),
)

(64, 1) (64,)


NameError: name 'target_dimension_grid' is not defined

In [ ]:
raise

In [ ]:
from sklearn.metrics import auc, roc_curve

In [ ]:
maths = (
    "P(S = 1 | G)",
    "P(S = 1 | A)",
    "P(S = 1 | F)",
    "P(S = 1 | A, F)",
    "P(S = 1 | A, F) (naive)",
)

math_roc = {math: {} for math in maths}

for math, p_s1__v, vs in zip(
    maths,
    (p_s1__g, p_s1__a, p_s1__f, p_s1__a_f, p_s1__a_f_naive),
    ((g,), (a,), (f,), (a, f), (a, f)),
):

    p_s1__vv = np.full(s.size, np.nan)

    for i in range(s.size):

        index_grid_values = tuple(
            [
                np.argmin(
                    np.absolute(np.linspace(v.min(), v.max(), num=n_grid) - v.values[i])
                )
            ]
            for v in vs
        )

        p_s1__vv[i] = p_s1__v[index_grid_values]

    fpr, tpr, t = roc_curve(s, kraft.normalize_nd_array(p_s1__vv, None, "0-1"))

    math_roc[math]["fpr"] = fpr

    math_roc[math]["tpr"] = tpr

    auc_ = auc(fpr, tpr)

    math_roc[math]["auc"] = auc_

    n_permutation_for_roc = 100

    permuting_aucs = np.full(n_permutation_for_roc, np.nan)

    permuting_s = s.values.copy()

    for i in range(n_permutation_for_roc):

        np.random.shuffle(permuting_s)

        permuting_fpr, permuting_tpr, permuting_t = roc_curve(permuting_s, p_s1__vv)

        permuting_aucs[i] = auc(permuting_fpr, permuting_tpr)

    math_roc[math]["p-value"] = kraft.compute_empirical_p_value(
        auc_, permuting_aucs, ">"
    )

In [ ]:
kraft.plot_bayesian_nomogram(
    s.values, 1, 0, n_grid, (p_s__g, p_s__a, p_s__f), (g.name, a.name, f.name)
)

In [ ]:
random_roc = np.linspace(0, 1, num=16)

kraft.plot_and_save(
    {
        "layout": {
            "title": {"text": "ROC<br>{}".format(", ".join((g.name, a.name, f.name)))},
            "xaxis": {"title": "False Positive Rate"},
            "yaxis": {"title": "True Positive Rate"},
        },
        "data": [
            {
                "type": "scatter",
                "name": "Random",
                "x": random_roc,
                "y": random_roc,
                "mode": "lines",
                "marker": {"color": "#d8d8d8"},
            },
            *(
                {
                    "type": "scatter",
                    "name": "{} | {:0.3f} | {:0.1e}".format(
                        math, math_roc[math]["auc"], math_roc[math]["p-value"]
                    ),
                    "x": math_roc[math]["fpr"],
                    "y": math_roc[math]["tpr"],
                    "mode": "markers + lines",
                }
                for math in maths
            ),
        ],
    },
    None,
)